In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import linregress
import math

In [20]:
#merging all the datasets into one
mergestorm_df = pd.concat( 
    map(pd.read_csv, ["resources/stormevents_1950.csv", "resources/stormevents_1951.csv", "resources/stormevents_1952.csv", 
                      "resources/stormevents_1953.csv", "resources/stormevents_1954.csv", "resources/stormevents_1955.csv",
                      "resources/stormevents_1956.csv", "resources/stormevents_1957.csv", "resources/stormevents_1958.csv",
                      "resources/stormevents_1959.csv", "resources/stormevents_1960.csv", "resources/stormevents_1961.csv",
                      "resources/stormevents_1962.csv", "resources/stormevents_1963.csv", "resources/stormevents_1964.csv"]),
                      ignore_index=True)
mergestorm_df.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,195004,28,1445,195004,28,1445,NaN,10096222,OKLAHOMA,40,...,0,NaN,NaN,35.12,-99.20,35.17,-99.20,NaN,NaN,PUB
1,195004,29,1530,195004,29,1530,NaN,10120412,TEXAS,48,...,0,NaN,NaN,31.90,-98.60,31.73,-98.60,NaN,NaN,PUB
2,195007,5,1800,195007,5,1800,NaN,10104927,PENNSYLVANIA,42,...,0,NaN,NaN,40.58,-75.70,40.65,-75.47,NaN,NaN,PUB
3,195007,5,1830,195007,5,1830,NaN,10104928,PENNSYLVANIA,42,...,0,NaN,NaN,40.60,-76.75,NaN,NaN,NaN,NaN,PUB
4,195007,24,1440,195007,24,1440,NaN,10104929,PENNSYLVANIA,42,...,0,NaN,NaN,41.63,-79.68,NaN,NaN,NaN,NaN,PUB


In [21]:
# Create a DataFrame with selected columns 
clean_mergestorm_df = mergestorm_df[["YEAR", 
                                     "STATE", 
                                     "EVENT_TYPE",  
                                     "INJURIES_DIRECT", 
                                     "INJURIES_INDIRECT",
                                     "DEATHS_DIRECT", 
                                     "DEATHS_INDIRECT",
                                     "DAMAGE_PROPERTY",
                                     "DAMAGE_CROPS", 
                                     "SOURCE", 
                                     "MAGNITUDE", 
                                     "MAGNITUDE_TYPE", 
                                     "TOR_F_SCALE", 
                                     "TOR_LENGTH",
                                     "TOR_WIDTH",
                                     "TOR_OTHER_CZ_STATE",
                                     "BEGIN_LOCATION", 
                                     "BEGIN_LAT", 
                                     "BEGIN_LON", 
                                     "END_LAT", 
                                     "END_LON", 
                                     "EPISODE_ID", 
                                     "EVENT_ID",]]
clean_mergestorm_df.head()

,YEAR,STATE,EVENT_TYPE,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,SOURCE,...,TOR_LENGTH,TOR_WIDTH,TOR_OTHER_CZ_STATE,BEGIN_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_ID,EVENT_ID
0,1950,OKLAHOMA,Tornado,0,0,0,0,250K,0,NaN,...,3.4,400,NaN,NaN,35.12,-99.20,35.17,-99.20,NaN,10096222
1,1950,TEXAS,Tornado,0,0,0,0,25K,0,NaN,...,11.5,200,NaN,NaN,31.90,-98.60,31.73,-98.60,NaN,10120412
2,1950,PENNSYLVANIA,Tornado,2,0,0,0,25K,0,NaN,...,12.9,33,NaN,NaN,40.58,-75.70,40.65,-75.47,NaN,10104927
3,1950,PENNSYLVANIA,Tornado,0,0,0,0,2.5K,0,NaN,...,0.0,13,NaN,NaN,40.60,-76.75,NaN,NaN,NaN,10104928
4,1950,PENNSYLVANIA,Tornado,0,0,0,0,2.5K,0,NaN,...,0.0,33,NaN,NaN,41.63,-79.68,NaN,NaN,NaN,10104929


In [22]:
clean_mergestorm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22087 entries, 0 to 22086
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   YEAR                22087 non-null  int64  
 1   STATE               22087 non-null  object 
 2   EVENT_TYPE          22087 non-null  object 
 3   INJURIES_DIRECT     22087 non-null  int64  
 4   INJURIES_INDIRECT   22087 non-null  int64  
 5   DEATHS_DIRECT       22087 non-null  int64  
 6   DEATHS_INDIRECT     22087 non-null  int64  
 7   DAMAGE_PROPERTY     22087 non-null  object 
 8   DAMAGE_CROPS        22087 non-null  int64  
 9   SOURCE              0 non-null      float64
 10  MAGNITUDE           22087 non-null  float64
 11  MAGNITUDE_TYPE      0 non-null      float64
 12  TOR_F_SCALE         7748 non-null   object 
 13  TOR_LENGTH          22087 non-null  float64
 14  TOR_WIDTH           22087 non-null  int64  
 15  TOR_OTHER_CZ_STATE  0 non-null      float64
 16  BEGI

In [23]:
clean_mergestorm_df.loc["DAMAGE_PROPERTY"] = clean_mergestorm_df["DAMAGE_PROPERTY"].fillna("0.00K")
clean_mergestorm_df.loc["DAMAGE_CROPS"] = clean_mergestorm_df["DAMAGE_CROPS"].fillna("0.00K")

C:\Users\lnfme\AppData\Local\Temp\ipykernel_35452\2794455276.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_mergestorm_df["DAMAGE_PROPERTY"] = clean_mergestorm_df["DAMAGE_PROPERTY"].fillna("0.00K")
C:\Users\lnfme\AppData\Local\Temp\ipykernel_35452\2794455276.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_mergestorm_df["DAMAGE_CROPS"] = clean_mergestorm_df["DAMAGE_CROPS"].fillna("0.00K")


In [25]:
clean_mergestorm_df['DAMAGE_CROPS'] = clean_mergestorm_df['DAMAGE_CROPS'].apply(retype_damage_value)
clean_mergestorm_df['DAMAGE_PROPERTY'] = clean_mergestorm_df['DAMAGE_PROPERTY'].apply(retype_damage_value)

C:\Users\lnfme\AppData\Local\Temp\ipykernel_35452\2980095712.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_mergestorm_df['DAMAGE_CROPS'] = clean_mergestorm_df['DAMAGE_CROPS'].apply(retype_damage_value)
C:\Users\lnfme\AppData\Local\Temp\ipykernel_35452\2980095712.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_mergestorm_df['DAMAGE_PROPERTY'] = clean_mergestorm_df['DAMAGE_PROPERTY'].apply(retype_damage_value)


In [26]:
clean_mergestorm_df

,YEAR,STATE,EVENT_TYPE,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,SOURCE,...,TOR_LENGTH,TOR_WIDTH,TOR_OTHER_CZ_STATE,BEGIN_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_ID,EVENT_ID
0,1950,OKLAHOMA,Tornado,0,0,0,0,250000.0,0,NaN,...,3.4,400,NaN,NaN,35.12,-99.20,35.17,-99.20,NaN,10096222
1,1950,TEXAS,Tornado,0,0,0,0,25000.0,0,NaN,...,11.5,200,NaN,NaN,31.90,-98.60,31.73,-98.60,NaN,10120412
2,1950,PENNSYLVANIA,Tornado,2,0,0,0,25000.0,0,NaN,...,12.9,33,NaN,NaN,40.58,-75.70,40.65,-75.47,NaN,10104927
3,1950,PENNSYLVANIA,Tornado,0,0,0,0,2500.0,0,NaN,...,0.0,13,NaN,NaN,40.60,-76.75,NaN,NaN,NaN,10104928
4,1950,PENNSYLVANIA,Tornado,0,0,0,0,2500.0,0,NaN,...,0.0,33,NaN,NaN,41.63,-79.68,NaN,NaN,NaN,10104929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22082,1964,KANSAS,Hail,0,0,0,0,0.0,0,NaN,...,0.0,0,NaN,NaN,39.38,-99.28,NaN,NaN,NaN,10025243
22083,1964,KANSAS,Tornado,2,0,0,0,0.0,0,NaN,...,25.7,880,NaN,NaN,37.95,-98.88,38.27,-98.63,NaN,10025244
22084,1964,KANSAS,Tornado,0,0,0,0,250000.0,0,NaN,...,34.0,880,NaN,NaN,38.27,-98.63,38.70,-98.33,NaN,10025245
22085,1964,KANSAS,Hail,0,0,0,0,0.0,0,NaN,...,0.0,0,NaN,NaN,37.88,-95.40,NaN,NaN,NaN,10025246


In [27]:
# =============================================
# change a string into a float; account for magnitude of
# # the number as represented in the string
def retype_damage_value(value_str):
    if pd.isna(value_str):
        return np.nan

    if isinstance(value_str, (int, float)):
        return value_str

    value_str = str(value_str)
    
    num_str = ''.join(char for char in value_str if char.isnumeric() or char == '.')
    magnitude = ''.join(char for char in value_str if char.isalpha())
    
    try:
        num = float(num_str)
    except ValueError:
        return np.nan
        
    if magnitude == 'K':
        num *= 1000
    elif magnitude == 'M':
        num *= 1000000
    elif magnitude == 'B':
        num *= 1000000000
    
    return num

In [28]:
clean_mergestorm_df['DAMAGE_PROPERTY'].apply(retype_damage_value)

0        250000.0
1         25000.0
2         25000.0
3          2500.0
4          2500.0
           ...   
22082         0.0
22083         0.0
22084    250000.0
22085         0.0
22086         0.0
Name: DAMAGE_PROPERTY, Length: 22087, dtype: float64

In [29]:
clean_mergestorm_df['DAMAGE_CROPS'].apply(retype_damage_value)

0        0
1        0
2        0
3        0
4        0
        ..
22082    0
22083    0
22084    0
22085    0
22086    0
Name: DAMAGE_CROPS, Length: 22087, dtype: int64

In [30]:
clean_mergestorm_df

,YEAR,STATE,EVENT_TYPE,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,SOURCE,...,TOR_LENGTH,TOR_WIDTH,TOR_OTHER_CZ_STATE,BEGIN_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_ID,EVENT_ID
0,1950,OKLAHOMA,Tornado,0,0,0,0,250000.0,0,NaN,...,3.4,400,NaN,NaN,35.12,-99.20,35.17,-99.20,NaN,10096222
1,1950,TEXAS,Tornado,0,0,0,0,25000.0,0,NaN,...,11.5,200,NaN,NaN,31.90,-98.60,31.73,-98.60,NaN,10120412
2,1950,PENNSYLVANIA,Tornado,2,0,0,0,25000.0,0,NaN,...,12.9,33,NaN,NaN,40.58,-75.70,40.65,-75.47,NaN,10104927
3,1950,PENNSYLVANIA,Tornado,0,0,0,0,2500.0,0,NaN,...,0.0,13,NaN,NaN,40.60,-76.75,NaN,NaN,NaN,10104928
4,1950,PENNSYLVANIA,Tornado,0,0,0,0,2500.0,0,NaN,...,0.0,33,NaN,NaN,41.63,-79.68,NaN,NaN,NaN,10104929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22082,1964,KANSAS,Hail,0,0,0,0,0.0,0,NaN,...,0.0,0,NaN,NaN,39.38,-99.28,NaN,NaN,NaN,10025243
22083,1964,KANSAS,Tornado,2,0,0,0,0.0,0,NaN,...,25.7,880,NaN,NaN,37.95,-98.88,38.27,-98.63,NaN,10025244
22084,1964,KANSAS,Tornado,0,0,0,0,250000.0,0,NaN,...,34.0,880,NaN,NaN,38.27,-98.63,38.70,-98.33,NaN,10025245
22085,1964,KANSAS,Hail,0,0,0,0,0.0,0,NaN,...,0.0,0,NaN,NaN,37.88,-95.40,NaN,NaN,NaN,10025246


In [33]:
#convert crop damages from object to string
clean_mergestorm_df["DAMAGE_CROPS"].astype(str)
clean_mergestorm_df["DAMAGE_PROPERTY"].astype(str)

0        250000.0
1         25000.0
2         25000.0
3          2500.0
4          2500.0
           ...   
22082         0.0
22083         0.0
22084    250000.0
22085         0.0
22086         0.0
Name: DAMAGE_PROPERTY, Length: 22087, dtype: object

In [34]:
clean_mergestorm_df["DAMAGE_CROPS"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 22087 entries, 0 to 22086
Series name: DAMAGE_CROPS
Non-Null Count  Dtype
--------------  -----
22087 non-null  int64
dtypes: int64(1)
memory usage: 172.7 KB


In [35]:
# Add new columns that combine direct/indirect deaths; direct/indirect injuries; property/crop damage
clean_mergestorm_df["TOTAL DEATHS"] = clean_mergestorm_df["DEATHS_DIRECT"] + clean_mergestorm_df["DEATHS_INDIRECT"]
clean_mergestorm_df["TOTAL INJURIES"] = clean_mergestorm_df["INJURIES_DIRECT"] + clean_mergestorm_df["INJURIES_INDIRECT"]
clean_mergestorm_df["TOTAL DAMAGES"] = clean_mergestorm_df["DAMAGE_PROPERTY"] + clean_mergestorm_df["DAMAGE_CROPS"]

clean_mergestorm_df

C:\Users\lnfme\AppData\Local\Temp\ipykernel_35452\387246326.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_mergestorm_df["TOTAL DEATHS"] = clean_mergestorm_df["DEATHS_DIRECT"] + clean_mergestorm_df["DEATHS_INDIRECT"]
C:\Users\lnfme\AppData\Local\Temp\ipykernel_35452\387246326.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_mergestorm_df["TOTAL INJURIES"] = clean_mergestorm_df["INJURIES_DIRECT"] + clean_mergestorm_df["INJURIES_INDIRECT"]
C:\Users\lnfme\AppData\Local\Temp\ipykernel_35452\

,YEAR,STATE,EVENT_TYPE,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,SOURCE,...,BEGIN_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_ID,EVENT_ID,TOTAL DEATHS,TOTAL INJURIES,TOTAL DAMAGES
0,1950,OKLAHOMA,Tornado,0,0,0,0,250000.0,0,NaN,...,NaN,35.12,-99.20,35.17,-99.20,NaN,10096222,0,0,250000.0
1,1950,TEXAS,Tornado,0,0,0,0,25000.0,0,NaN,...,NaN,31.90,-98.60,31.73,-98.60,NaN,10120412,0,0,25000.0
2,1950,PENNSYLVANIA,Tornado,2,0,0,0,25000.0,0,NaN,...,NaN,40.58,-75.70,40.65,-75.47,NaN,10104927,0,2,25000.0
3,1950,PENNSYLVANIA,Tornado,0,0,0,0,2500.0,0,NaN,...,NaN,40.60,-76.75,NaN,NaN,NaN,10104928,0,0,2500.0
4,1950,PENNSYLVANIA,Tornado,0,0,0,0,2500.0,0,NaN,...,NaN,41.63,-79.68,NaN,NaN,NaN,10104929,0,0,2500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22082,1964,KANSAS,Hail,0,0,0,0,0.0,0,NaN,...,NaN,39.38,-99.28,NaN,NaN,NaN,10025243,0,0,0.0
22083,1964,KANSAS,Tornado,2,0,0,0,0.0,0,NaN,...,NaN,37.95,-98.88,38.27,-98.63,NaN,10025244,0,2,0.0
22084,1964,KANSAS,Tornado,0,0,0,0,250000.0,0,NaN,...,NaN,38.27,-98.63,38.70,-98.33,NaN,10025245,0,0,250000.0
22085,1964,KANSAS,Hail,0,0,0,0,0.0,0,NaN,...,NaN,37.88,-95.40,NaN,NaN,NaN,10025246,0,0,0.0


In [36]:
drop_merged_df = clean_mergestorm_df.drop(columns=["INJURIES_DIRECT", 
                                                   "INJURIES_INDIRECT",
                                                   "DEATHS_DIRECT", 
                                                   "DEATHS_INDIRECT"])
drop_merged_df.head()

,YEAR,STATE,EVENT_TYPE,DAMAGE_PROPERTY,DAMAGE_CROPS,SOURCE,MAGNITUDE,MAGNITUDE_TYPE,TOR_F_SCALE,TOR_LENGTH,...,BEGIN_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_ID,EVENT_ID,TOTAL DEATHS,TOTAL INJURIES,TOTAL DAMAGES
0,1950,OKLAHOMA,Tornado,250000.0,0,NaN,0.0,NaN,F3,3.4,...,NaN,35.12,-99.20,35.17,-99.20,NaN,10096222,0,0,250000.0
1,1950,TEXAS,Tornado,25000.0,0,NaN,0.0,NaN,F1,11.5,...,NaN,31.90,-98.60,31.73,-98.60,NaN,10120412,0,0,25000.0
2,1950,PENNSYLVANIA,Tornado,25000.0,0,NaN,0.0,NaN,F2,12.9,...,NaN,40.58,-75.70,40.65,-75.47,NaN,10104927,0,2,25000.0
3,1950,PENNSYLVANIA,Tornado,2500.0,0,NaN,0.0,NaN,F2,0.0,...,NaN,40.60,-76.75,NaN,NaN,NaN,10104928,0,0,2500.0
4,1950,PENNSYLVANIA,Tornado,2500.0,0,NaN,0.0,NaN,F0,0.0,...,NaN,41.63,-79.68,NaN,NaN,NaN,10104929,0,0,2500.0


In [37]:
# Narrow down events that have had at least one death and/or at least one injury 
narrow_df = drop_merged_df.loc[(drop_merged_df["TOTAL DEATHS"] > 1) | (drop_merged_df["TOTAL INJURIES"] > 1)]

# reset the index 
narrow_df.reset_index(drop=True, inplace=True)

# Display sample data
narrow_df.head()

,YEAR,STATE,EVENT_TYPE,DAMAGE_PROPERTY,DAMAGE_CROPS,SOURCE,MAGNITUDE,MAGNITUDE_TYPE,TOR_F_SCALE,TOR_LENGTH,...,BEGIN_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_ID,EVENT_ID,TOTAL DEATHS,TOTAL INJURIES,TOTAL DAMAGES
0,1950,PENNSYLVANIA,Tornado,25000.0,0,NaN,0.0,NaN,F2,12.9,...,NaN,40.58,-75.70,40.65,-75.47,NaN,10104927,0,2,25000.0
1,1950,OKLAHOMA,Tornado,250000.0,0,NaN,0.0,NaN,F2,6.8,...,NaN,35.00,-96.25,35.07,-96.17,NaN,10099490,0,6,250000.0
2,1950,OKLAHOMA,Tornado,250000.0,0,NaN,0.0,NaN,F4,4.5,...,NaN,35.08,-96.40,35.13,-96.35,NaN,10096223,5,32,250000.0
3,1950,OKLAHOMA,Tornado,25000.0,0,NaN,0.0,NaN,F2,4.3,...,NaN,36.12,-97.83,36.17,-97.78,NaN,10096230,0,3,25000.0
4,1950,MISSISSIPPI,Tornado,25000.0,0,NaN,0.0,NaN,F1,0.1,...,NaN,34.73,-88.27,NaN,NaN,NaN,10049534,0,2,25000.0


In [38]:
reordernarrow_df = narrow_df[["YEAR", 
                              "STATE", 
                              "EVENT_TYPE", 
                              "TOTAL DEATHS", 
                              "TOTAL INJURIES", 
                              "DAMAGE_PROPERTY",
                              "DAMAGE_CROPS", 
                              "SOURCE", 
                              "MAGNITUDE", 
                              "MAGNITUDE_TYPE", 
                              "TOR_F_SCALE", 
                              "TOR_LENGTH", 
                              "TOR_WIDTH", 
                              "TOR_OTHER_CZ_STATE", 
                              "BEGIN_LOCATION", 
                              "BEGIN_LAT", 
                              "BEGIN_LON", 
                              "END_LAT", 
                              "END_LON", 
                              "EPISODE_ID", 
                              "EVENT_ID",]]
reordernarrow_df.head()

,YEAR,STATE,EVENT_TYPE,TOTAL DEATHS,TOTAL INJURIES,DAMAGE_PROPERTY,DAMAGE_CROPS,SOURCE,MAGNITUDE,MAGNITUDE_TYPE,...,TOR_LENGTH,TOR_WIDTH,TOR_OTHER_CZ_STATE,BEGIN_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_ID,EVENT_ID
0,1950,PENNSYLVANIA,Tornado,0,2,25000.0,0,NaN,0.0,NaN,...,12.9,33,NaN,NaN,40.58,-75.70,40.65,-75.47,NaN,10104927
1,1950,OKLAHOMA,Tornado,0,6,250000.0,0,NaN,0.0,NaN,...,6.8,100,NaN,NaN,35.00,-96.25,35.07,-96.17,NaN,10099490
2,1950,OKLAHOMA,Tornado,5,32,250000.0,0,NaN,0.0,NaN,...,4.5,200,NaN,NaN,35.08,-96.40,35.13,-96.35,NaN,10096223
3,1950,OKLAHOMA,Tornado,0,3,25000.0,0,NaN,0.0,NaN,...,4.3,33,NaN,NaN,36.12,-97.83,36.17,-97.78,NaN,10096230
4,1950,MISSISSIPPI,Tornado,0,2,25000.0,0,NaN,0.0,NaN,...,0.1,10,NaN,NaN,34.73,-88.27,NaN,NaN,NaN,10049534


In [39]:
groupbynarrow_df=reordernarrow_df.sort_values(["YEAR","TOTAL DEATHS"], ascending=True).groupby("TOTAL DEATHS").head()
groupbynarrow_df.reset_index(drop=True, inplace=True)
groupbynarrow_df

,YEAR,STATE,EVENT_TYPE,TOTAL DEATHS,TOTAL INJURIES,DAMAGE_PROPERTY,DAMAGE_CROPS,SOURCE,MAGNITUDE,MAGNITUDE_TYPE,...,TOR_LENGTH,TOR_WIDTH,TOR_OTHER_CZ_STATE,BEGIN_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_ID,EVENT_ID
0,1950,PENNSYLVANIA,Tornado,0,2,25000.0,0,NaN,0.0,NaN,...,12.9,33,NaN,NaN,40.58,-75.70,40.65,-75.47,NaN,10104927
1,1950,OKLAHOMA,Tornado,0,6,250000.0,0,NaN,0.0,NaN,...,6.8,100,NaN,NaN,35.00,-96.25,35.07,-96.17,NaN,10099490
2,1950,OKLAHOMA,Tornado,0,3,25000.0,0,NaN,0.0,NaN,...,4.3,33,NaN,NaN,36.12,-97.83,36.17,-97.78,NaN,10096230
3,1950,MISSISSIPPI,Tornado,0,2,25000.0,0,NaN,0.0,NaN,...,0.1,10,NaN,NaN,34.73,-88.27,NaN,NaN,NaN,10049534
4,1950,TEXAS,Tornado,0,2,0.0,0,NaN,0.0,NaN,...,4.7,133,NaN,NaN,26.88,-98.12,26.88,-98.05,NaN,10120401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,1960,OKLAHOMA,Tornado,13,100,2500000.0,0,NaN,0.0,NaN,...,17.2,200,NaN,NaN,34.88,-95.30,34.63,-95.30,NaN,10093986
76,1961,TEXAS,Tornado,8,200,0.0,0,NaN,0.0,NaN,...,1.0,100,NaN,NaN,29.32,-94.78,NaN,NaN,NaN,10120340
77,1961,OKLAHOMA,Tornado,16,58,250000.0,0,NaN,0.0,NaN,...,26.4,400,NaN,NaN,34.73,-95.03,34.95,-94.65,NaN,10092640
78,1962,FLORIDA,Tornado,17,100,2500000.0,0,NaN,0.0,NaN,...,6.9,440,NaN,NaN,30.62,-87.10,30.67,-87.00,NaN,9987256


In [40]:
# Export the narrowed data into a csv
groupbynarrow_df.to_csv("groupbynarrow_df.csv")